# ROC on openpifpaf

In [1]:
from mlworkflow import PickledDataset, TransformedDataset
from dataset_utilities.ds.instants_dataset import ExtractViewData, ViewCropperTransform

from openpifpaf.datasets.deepsport import AddBallSegmentationTargetViewFactory, AddBallPositionFactory

ds = PickledDataset("/scratch/gva/views_camera_with_ball2.pickle")

shape = (800,600)

ds = TransformedDataset(ds, [
    ViewCropperTransform(def_min=30, def_max=80, output_shape=shape),
    ExtractViewData(AddBallPositionFactory(), AddBallSegmentationTargetViewFactory()),
])

/import/home/gva/poseestimation_emb/openpifpaf/visualizer/base.py:12: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("Oranges"))
  matplotlib.cm.get_cmap('Oranges').set_bad('white', alpha=0.5)
/import/home/gva/poseestimation_emb/openpifpaf/visualizer/base.py:13: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("Blues"))
  matplotlib.cm.get_cmap('Blues').set_bad('white', alpha=0.5)
/import/home/gva/poseestimation_emb/openpifpaf/visualizer/base.py:14: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colorma

In [2]:
import random
keys = list(PickledDataset("/scratch/gva/camera_views_with_human_masks.pickle").keys.all())
random.seed(0)
random.shuffle(keys)
validation_set_size_pc = 15
lim = len(keys)*validation_set_size_pc//100
training_keys = keys[lim:]
validation_keys = keys[:lim]

In [3]:
print(len(validation_keys))
validation_keys = [k for k in ds.yield_keys() if k not in training_keys]
print(len(validation_keys))
print(len(training_keys))

50
1394
286


In [6]:
import tensorflow as tf
import numpy as np
from tf_layers import AvoidLocalEqualities, PeakLocalMax, ComputeElementaryMetrics
class ChunkProcessor:
    pass

class CastFloat(ChunkProcessor):
    def __init__(self, tensor_name):
        self.tensor_name = [tensor_name] if isinstance(tensor_name, str) else tensor_name
    def __call__(self, chunk):
        for tensor_name in self.tensor_name:
            if tensor_name in chunk:
                chunk[tensor_name] = tf.cast(chunk[tensor_name], tf.float32)
class Normalize(ChunkProcessor):
    def __init__(self, tensor_name):
        self.tensor_name = tensor_name
    def __call__(self, chunk):
        assert chunk[self.tensor_name].dtype == tf.float32
        chunk[self.tensor_name] = chunk[self.tensor_name]/255
class ComputeKeypointsDetectionAccuracy(ChunkProcessor):
    def __init__(self, non_max_suppression_pool_size=50, threshold=0.5, target_enlargment_size=10):
        thresholds = threshold if isinstance(threshold, np.ndarray) else np.array([threshold])
        assert len(thresholds.shape) == 1, "'threshold' argument should be 1D-array (a scalar is also accepted)."

        self.avoid_local_eq = AvoidLocalEqualities()
        self.peak_local_max = PeakLocalMax(min_distance=non_max_suppression_pool_size//2, thresholds=thresholds)
        self.enlarge_target = tf.keras.layers.MaxPool2D(target_enlargment_size, strides=1, padding="same")
        self.compute_metric = ComputeElementaryMetrics()

    def __call__(self, chunk):
        batch_target = tf.cast(chunk["batch_target"], tf.float32)
        batch_target = batch_target if len(batch_target.shape) == 4 else batch_target[...,tf.newaxis]
        batch_output = chunk["batch_heatmap"]
        batch_output = batch_output if len(batch_output.shape) == 4 else batch_output[...,tf.newaxis]

        batch_output = self.avoid_local_eq(batch_output)
        batch_hitmap = self.peak_local_max(batch_output)
        batch_hitmap = tf.cast(batch_hitmap, tf.int32)
        chunk["batch_hitmap"] = tf.squeeze(batch_hitmap)
        batch_target = self.enlarge_target(batch_target)
        batch_target = tf.cast(batch_target, tf.int32)[..., tf.newaxis]

        batch_metric = self.compute_metric(batch_hitmap=batch_hitmap, batch_target=batch_target)
        chunk["batch_TP"] = tf.squeeze(batch_metric["batch_TP"])
        chunk["batch_FP"] = tf.squeeze(batch_metric["batch_FP"])
        chunk["batch_TN"] = tf.squeeze(batch_metric["batch_TN"])
        chunk["batch_FN"] = tf.squeeze(batch_metric["batch_FN"])

        
chunk = {}
chunk["batch_heatmap"] = tf.keras.Input(dtype=tf.uint8, shape=(shape[1], shape[0]), name="batch_heatmap")
chunk["batch_target"] = tf.keras.Input(dtype=tf.uint8, shape=(shape[1], shape[0]), name="batch_target")
inputs = dict(chunk) # makes a copy

thresholds = np.array([])
n_points = 21
chunk_processors = [
    CastFloat(["batch_heatmap", "batch_target"]),
    Normalize("batch_heatmap"),
    ComputeKeypointsDetectionAccuracy(non_max_suppression_pool_size=20, threshold=np.linspace(0,1,n_points)),
]
for cp in chunk_processors:
    cp(chunk)

outputs = {k:chunk[k] for k in chunk if k in ["batch_TP", "batch_TN", "batch_FP", "batch_FN"]}
model = tf.keras.Model(inputs, outputs)

In [8]:
from tqdm.notebook import tqdm
from openpifpaf.predict import main
import sys
import imageio
import numpy as np

from matplotlib import pyplot as plt
from ipywidgets import Output
from IPython import display


output = Output()
display.display(output)

#weights_file = "shufflenetv2k16w-210225-164102-ball-edge501.pkl.epoch150" # trained on small dataset
#weights_file = "shufflenetv2k16w-210308-164747-ball-edge501.pkl.epoch150" # trained on full dataset
#weights_file = "shufflenetv2k16-210309-153430-ball-edge501.pkl.epoch200" # trained on small dataset with fixed seed
weights_file = "shufflenetv2k16-210309-220522-ball.pkl.epoch499" # trained on small dataset with fixed seed
result = {}
for set_name in ["training_keys", "validation_keys"]:
    resul = result[set_name] = {}
    resul["TP"] = np.zeros(n_points, np.int32)
    resul["FP"] = np.zeros(n_points, np.int32)
    resul["TN"] = np.zeros(n_points, np.int32)
    resul["FN"] = np.zeros(n_points, np.int32)
    for key in tqdm(training_keys):
        data = ds.query_item(key)
        filename = "/home/gva/test.png"
        imageio.imwrite(filename, data["input_image"])
        sys.argv = [
            "aue",
            filename,
            "--checkpoint", f"/home/gva/poseestimation_emb/results/{weights_file}",
            "--image-output",
            "--debug-images", "--debug-cif-c", "--debug"
        ]
        main()
        heatmap = imageio.imread("/home/gva/test.accumulated.png")
        result  = model({"batch_heatmap": heatmap[np.newaxis], "batch_target": data["mask"][np.newaxis]})
        resul["TP"] += result["batch_TP"].numpy()
        resul["FP"] += result["batch_FP"].numpy()
        resul["TN"] += result["batch_TN"].numpy()
        resul["FN"] += result["batch_FN"].numpy()
        with output:
            fig = plt.figure()
            display.clear_output(wait=True)
            x = resul["FP"]/(resul["FP"]+resul["TN"])
            y = resul["TP"]/(resul["TP"]+resul["FN"])
            ax = fig.gca()
            
            index = 10
            
            ax.plot(x,y, linestyle="-", linewidth=1, markersize=5, marker=".", label=set_name)
            ax.plot(x[index], y[index], markersize=10, marker=".", color="green")
            ax.text(x[index]+0.05, y[index]-0.05, "threshold={}".format(np.linspace(0,1,n_points)[index]), color="green")
            ax.set_xlabel("FP rate")
            ax.set_ylabel("TP rate")
            ax.axis("equal")
            ax.set_xlim([0,1])
            ax.set_ylim([0,1])
            ax.set_box_aspect(1)
            ax.set_title(f"ROC ball detection from PIFPAF")
            ax.legend()
            
            display.display(fig)
            plt.close()


Output()

  0%|          | 0/286 [00:00<?, ?it/s]

DEBUG:openpifpaf.predict:neural network device: cuda
DEBUG:openpifpaf.network.heads:cif = [0], caf = []


RuntimeError: CUDA error: out of memory

In [ ]:
%matplotlib inline
fig = plt.figure()
ax = fig.gca()

for name in ["training_set", "validation_set"]:
    d = eval(name)
    x = d["FP"]/(d["FP"]+d["TN"])
    y = d["TP"]/(d["TP"]+d["FN"])
    index = 10
    ax.plot(x,y, linestyle="-", linewidth=1, markersize=5, marker=".", label=name)
    ax.plot(x[index], y[index], markersize=10, marker=".", color="green")
    ax.text(x[index]+0.05, y[index]-0.05, "threshold={}".format(np.linspace(0,1,n_points)[index]), color="green")
    ax.set_xlabel("FP rate")
    ax.set_ylabel("TP rate")
    ax.axis("equal")
    ax.set_xlim([0,1])
    ax.set_ylim([0,1])
    ax.set_box_aspect(1)
    ax.set_title(f"ROC ball detection from PIFPAF")
    ax.legend()
plt.show()